# AI&데이터 마이닝 7, 8일차 (2020.08.02)

# 07. 회귀와 분류
## 예측 개념 소개
### 예측??
- 보이지 않는 미래의 데이터에 대해 출력값을 예측(회귀, regression)하거나 레이블을 예측(분류, classification)
- 주요 전략
    - 훈련 데이터 집합 수집
    - 입력 독립변수(X)를 출력 종속변수(Y)로 매핑하는 함수 구축
- 가정 1) X와 Y 사이에는 연관성이 있음 : Y는 독립변수 X에 의존적
- 가정 2) 미래 데이터는 훈련 데이터 집합과 같은 분포를 가짐


### 매핑(Mapping) 함수 = 가설(hypothesis) 함수
- 선형 / 버선형 관계 모델링
- 다수의 내부 파라미터 가지며 최적의 적합(fit, 훈련을 통한 모델 생성)을 위해서는 반드시 최적화 되어야 함
- 최적화 전략 : 예측 알고리즘을 최소화 문제로 수식화
    - 손실(loss) : 예측 실패를 수치화한 것
    - 손실 함수(loss function) : 손실 계산을 위한 함수
        - 수집한 데이터(X, Y)의 독립변수 X에 대한 예측 출력(Y_pred)과 Y를 비교
    - 최적화 == 손실 최소화 ?
        - 가장 일반적인 방법은 Gradient Descent

## 손실 함수와 기울기 하강
### 손실 함수(Loss Function)
- 모델의 예측 손실을 계산하는 수학식
- 예측 모델별로 서로 다른 함수 적용

### 선형회귀(Linear Regression) 예
y_pred = h(theta)(x) = (theta_1)x + (theta_0)
- x : X의 한 레코드
- h : hypothesis 함수
- (theta_1), (theta_2) : 각각의 기울기와 절편

### 기울기 하강 (Gradient Descent)
- 손실을 최소화하기 위한 손실함수의 파라미터 최적화
- 미분(derivative)이용
    - 미분값이 음수이면 오른쪽으로 이동 -> theta를 크게
    - 미분값이 양수이면 왼쪽으로 이동 -> theta를 작게
    - learning rate : step size가 너무 크면 최솟값을 지나갈 수 있고, 너무 작으면 시간이 너무 많이 걸림

## 예측 모델 품질
- 훈련 데이터 집합에 대한 예측 정확도 및 테스트 데이터 집합(훈련에 사용되지 않은 데이터 집합)에 대한 예측 정확도를 측정
- 고분산(High variance) : 과소적합(Underfit)
- 고편향(High bias) : 과적합(Overfit)

## 회귀(Regression)
### 개요
- 입력 데이터를 숫자값(예측값)으로 매핑
- 집값, 멀리뛰기 거리, 홈런 숫자 등 예측 가능

### 모델 예측 지표
- MSE(Mean Squared Error) : 선형회귀의 손실함수<br>
MSE = 1/m(시그마(y_pred - y_i)^2)
- 큰 입력데이터에는 큰 MSE 값, 작은 입력데이터에는 작은 MSE 값 -> 복수 케이스에 대한 평가는 어려움
    - 주택가격 예측 : 수억 ~ 수백억
    - 멀리뛰기 예측 : 수십m
- MSE 정규화
    - 주택가격 예측 지표 0.8과 멀리뛰기 예측 지표 0.8은 같은 수준의 예측
    - 결정 계수(Coefficient Determination)<br>
    R^2 Score = 1 - (MSE/분산)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_context("paper", font_scale=1.5)
sns.set_style("white")

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

dataset = load_boston()
print(dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [4]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

def get_boston():
    dataset = load_boston()
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    df['MEDV'] = dataset.target
    df.index.name = 'record'
    
    # print(df.head())
    
    X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'MEDV'], df['MEDV'], test_size=.33, random_state=42)
    
    return [X_train, X_test, y_train, y_test]

## Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = get_boston()

clf = LinearRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('r2 score is = ' + str(r2))

r2 score is = 0.7261570836552478


## 다변량 형식으로 확장
### 개요
- 대부분의 데이터 집합은 하나 이상의 입력 속성을 가짐
- 가설 함수를 다수 변수로 확장할 필요 있음 -> 행렬 이용 <br>
y_pred = h(theta)(x) = (theta_1)x + (theta_0)
- X : 속성 행렬, (theta) : 파라미터 행렬(theta_0, theta_1, theta_2, ...)

## 페널티 회귀
### 개념
- 전통적인 선형 회귀는 많은 가정을 전제로 하고 있음
    - 독립 변수와 종속 변수가 선형 관계 -> Polynomial regression, Generalized Additive Model(GAM)
    - 오차항의 확률분포가 정규분포 -> Generalized Linear Model(GLM)
    - 오차항에 자기 상관성이 없음 -> Auto-regression
    - 데이터에 아웃라이어가 없음 -> Robust regression, Quantile regression
    - 독립변수 간에 상관성이 없음 -> Ridge regression, Lasso regression, Elastic Net regression, Principal Component REgression (PCR), Partial Least Square(PLS) regression
- 다중 공신성
    - 다변량 회귀분석시 독립변수들 간에 상관관계가 존재
    - 독립변수들간의 상관관계가 강하면 각 독립변수의 설명력이 떨어져서 회귀 분석이 제대로 수행되지 않음
    - 해결 방안
        - 상관관계가 높은 독립변수 제거
        - 손실함수에 페널티를 추가하여 파라미터 영향력 축소 : **Ridge regression, Lasso regression**

## Lasso Regression

In [6]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = get_boston()

clf = Lasso(alpha=0.3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('r2 score is = ' + str(r2))

r2 score is = 0.7050894840749293


## Ridge Regression

In [7]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = get_boston()

clf = Ridge(alpha=0.3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print('r2 score is = ' + str(r2))

r2 score is = 0.7241938555718039


MEDV와 다른 features가 상관관계가 높지 않아 r2 score가 낮게 나옴(Linear Regression과 비교)

## 분류(Classification)
### 개요
- 입력 데이터를 카테고리 클래스 레이블로 예측
- 참/거짓, 저위험/중위험/고위험, 동물의 종
- Logistic Regression
- Decision Tree
- Random Forest
- Linear SVM / RBF SVM

### 분류 모델 예측 지표
- 정확도 : 예측한 것 중 예측이 맞은 비율
- 정밀도(Precision) : 예측한 것 중 True의 비율  TP / TP+FP
- 재현율(Recall) : True라고 예측한 것이 실제로 True인 비율  TP / TP+FN
- F1점수 : 정밀도와 재현율을 모두 고려  2 * ((Precision x Recall) / (Precision + Recall))
- 예) 그룹A(정상, Negative) 90개, 그룹B(암환자, Positive) 10개
    - 100번의 예측을 모두 A로 예측 : 90% 정확
    - 100번의 예측을 모두 B로 예측 : 10% 정확

### 복수 클래스 분류
- 레이블이 3개 이상인 경우의 분류 방법
- One vs. All, One vs One
- One-vs-rest(be built = 4)
    - [A] vs [B, C, D], [B] vs [A, C, D], [C] vs [A, B, D], [D] vs [A, B, C}
    - 각 분류 모델은 모든 레이블의 데이터로 훈련
    - 각 분류 모델로 예측을 수행 후 가장 점수가 높은 것 선택
- One-vs-one(be built = 6)
    - [A] vs [B], [A] vs [C], [A] vs [D], [B] vs [C], [B] vs [D], [C] vs [D]
    - 각 분류 모델은 해당 레이블의 데이터로만 훈련
    - 각 분류 모델로 예측을 수행하고 가장 많이 예측된 레이블을 선택
    
### 분류 예제 데이터 집합
- make_moons : 초승달 모양 클러스터 두개의 형상
- make_blobs : 동방성 가우시안 정규분포

In [8]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

def get_moon_data():
    X, y = make_moons(n_samples=150, noise=0.4, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)
    
    return [X_train, X_test, y_train, y_test]

## Logistic Regression
### 개요
- 회귀 기법이지만 실제로는 분류 기법
- 가설함수로 시그모이드(sigmoid)함수를 사용

### 가설함수
- 분류를 위해서는 Z = (theta_1)x + (theta_0) 일 때 x에 대한 가설함수의 출력이 0 또는 1이 되도록 해야 함
- Sigmoid 함수 이용 <br>
y_pred = h(theta)(Z) = sigmoid(Z)

### 손실함수 J
- y가 1일때 h(theta)가 1에 가까운 값을 출력하면 J는 0에 가까워지고, (  loss = -log(h(theta))  )
- y가 0일때 h(theta)가 0에 가까운 값을 출력하면 J가 1에 가까운 값을 갖도록 설계 (  loss = -log(1-h(theta))  )



## Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = get_moon_data()

clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print('f1 score is = ' + str(f1))

f1 score is = 0.7499999999999999


- Solver : 최적화 방법
    - newton-cg, liblinear, sab, saga
    - lbfgs : 가장 일반적, L2 penalty 제공

In [10]:
clf = LogisticRegression(solver='lbfgs', penalty='l2', C=0.5)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print('f1 score is = ' + str(f1_score(y_test, y_pred)))


f1 score is = 0.7499999999999999


## SVM(Support Vector Machine)
### 개요
- 로지스틱 회귀 모델의 확장
- 두 클래스 데이터들 간의 최대 마진(margin)을 찾아내는 것이 목표
- 선형 분류 뿐 아니라 커널 트릭(Kernel Trick)을 이용해 비선형 분류도 가능

### 소프트 마진 (C)
- 마진을 너무 strict 하게 유지하면 과적합 현상 발생
- 일부 오 분류를 허용하는 마진을 찾도록 C 값을 조정하는 것

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = get_moon_data()

clf = SVC(kernel='linear', C=0.5)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print('f1 score is = ' + str(f1))

f1 score is = 0.7499999999999999


### 커널 트릭(Kernel Trick)
- 비선형 분류를 위해 데이터를 다른 차원으로 매핑하는 것
- 가우시안 (Gaussian) 커널을 이용하여 gamma를 이용해 매핑 정도를 조절

In [12]:
clf = SVC(gamma=2, C=1) # default = gaussian
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print('f1 score is = ' + str(f1))

f1 score is = 0.782608695652174


## 트리 기반 분류 (Tree-based Classification)
### 개요
- 인간이 의사 결정하는 과정과 유사한 방식, 직관적
- 특별한 처리 없이 다중 클래스에 대한 분류가 용이
- 수치 데이터, 범주형 데이터 모두 처리가 가능하며 범주형 데이터는 one-hot encoding 등의 방법을 이용해서 전처리 필요

### 주요 방법
- 의사결정 트리 (Decision Tree)
    - 직관적이며 사람이 분류과정을 이해하기 쉬움
    - 트리가 복잡해지면 과적합 문제 발생
- 랜덤 포레스트 (Random Forest)
    - 여러 개의 작은 트리를 조합(ensemble)하여 분류
    - 과적합 문제 해결

## 의사 결정 트리 (Decision Tree)
### 개요
- 인간이 의사결정하는 방식과 유사
- 루트 노드는 모든 데이터 포함하며 분류 기준에 따라 자식 노드로 분할됨
### Gini 기반 노드 분할
- 어떤 기준으로 어떻게 분할하는지가 가장 중요
- 불순도(impurity) :
    - 현재 노드의 예측 신뢰도, 얼마나 서로 다른 클래스의 데이터가 섞여 있는지를 나타내는 지표
    - 불순도가 높으면 예측 신뢰도가 낮음
- 노드의 분할은 자식 노드의 불순도 합이 최소가 되는 방향으로<br>
location to split node = min(Imp_child1 + Imp_child2)
- 불순도의 수치화 : Gini, Entropy
- Gini 불순도
    - j : 클래스의 수
    - p_i : 어떤 데이터의 i번째 클래스에 속할 확률<br>
    Gini imp = 1 - (1 to j sigma ( p_i)^2 ))

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = get_moon_data()

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print('f1 score is = ' + str(f1))

f1 score is = 0.7391304347826089


## 랜덤 포레스트 (Random Forest)
### 개요
- 크기가 작고 과소적합된 다수의 트리가 ensemble 된 분류 기법
- 튜닝이 크게 필요 없으며, 과적합 문제가 발생하지 않음
- 다수 트리에 대해 분류를 시도하여 다수 분류결과에 대한 투표로 최종 분류 결과 확정

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = get_moon_data()

clf = RandomForestClassifier(max_depth=4, n_estimators=4, max_features='sqrt', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print('f1 score is = ' + str(f1))

f1 score is = 0.7755102040816326
